In [1]:
import pandas as pd 
import numpy as num
import matplotlib.pyplot as plt 
import seaborn as sns 

In [4]:
df = pd.read_csv(r"C:\Users\Administrator\OneDrive\Desktop\ml\diet recommendation system\data\diet_recommendations_dataset.csv")
df.head()

,Patient_ID,Age,Gender,Weight_kg,Height_cm,BMI,Disease_Type,Severity,Physical_Activity_Level,Daily_Caloric_Intake,Cholesterol_mg/dL,Blood_Pressure_mmHg,Glucose_mg/dL,Dietary_Restrictions,Allergies,Preferred_Cuisine,Weekly_Exercise_Hours,Adherence_to_Diet_Plan,Dietary_Nutrient_Imbalance_Score,Diet_Recommendation
0,P0001,56,Male,58.4,160,22.8,Obesity,Moderate,Moderate,3079,173.3,133,116.3,NaN,Peanuts,Mexican,3.1,96.6,3.1,Balanced
1,P0002,69,Male,101.2,169,35.4,Diabetes,Mild,Moderate,3032,199.2,120,137.1,NaN,Peanuts,Chinese,4.5,63.2,0.6,Low_Carb
2,P0003,46,Female,63.5,173,21.2,Hypertension,Mild,Sedentary,1737,181.0,121,109.6,NaN,Peanuts,Chinese,3.8,57.5,4.6,Low_Sodium
3,P0004,32,Male,58.1,164,21.6,NaN,Mild,Moderate,2657,168.2,144,159.4,NaN,NaN,Mexican,4.3,54.5,0.4,Balanced
4,P0005,60,Male,79.5,197,20.5,Diabetes,Moderate,Sedentary,3496,200.4,172,182.3,Low_Sugar,NaN,Italian,9.8,78.2,4.7,Low_Carb


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import joblib

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Patient_ID                        1000 non-null   object 
 1   Age                               1000 non-null   int64  
 2   Gender                            1000 non-null   object 
 3   Weight_kg                         1000 non-null   float64
 4   Height_cm                         1000 non-null   int64  
 5   BMI                               1000 non-null   float64
 6   Disease_Type                      796 non-null    object 
 7   Severity                          1000 non-null   object 
 8   Physical_Activity_Level           1000 non-null   object 
 9   Daily_Caloric_Intake              1000 non-null   int64  
 10  Cholesterol_mg/dL                 1000 non-null   float64
 11  Blood_Pressure_mmHg               1000 non-null   int64  
 12  Glucose

In [9]:
df.drop(['BMI','Daily_Caloric_Intake','Cholesterol_mg/dL','Blood_Pressure_mmHg','Glucose_mg/dL','Preferred_Cuisine'], axis=1, inplace=True)

In [10]:
df.drop(['Weekly_Exercise_Hours','Adherence_to_Diet_Plan','Dietary_Nutrient_Imbalance_Score'], axis=1, inplace=True)

In [11]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Weight_kg', 'Height_cm', 'Disease_Type',
       'Severity', 'Physical_Activity_Level', 'Dietary_Restrictions',
       'Allergies', 'Diet_Recommendation'],
      dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Patient_ID               1000 non-null   object 
 1   Age                      1000 non-null   int64  
 2   Gender                   1000 non-null   object 
 3   Weight_kg                1000 non-null   float64
 4   Height_cm                1000 non-null   int64  
 5   Disease_Type             796 non-null    object 
 6   Severity                 1000 non-null   object 
 7   Physical_Activity_Level  1000 non-null   object 
 8   Dietary_Restrictions     666 non-null    object 
 9   Allergies                677 non-null    object 
 10  Diet_Recommendation      1000 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 86.1+ KB


In [13]:
df = df.drop(columns=["Patient_ID"])


In [14]:
df["Disease_Type"] = df["Disease_Type"].fillna("None")
df["Dietary_Restrictions"] = df["Dietary_Restrictions"].fillna("None")
df["Allergies"] = df["Allergies"].fillna("None")


In [16]:
target_cols = "Diet_Recommendation"

X = df.drop(columns=[target_cols])
y = df[target_cols]

In [17]:
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

numeric_cols, categorical_cols


(['Age', 'Weight_kg', 'Height_cm'],
 ['Gender',
  'Disease_Type',
  'Severity',
  'Physical_Activity_Level',
  'Dietary_Restrictions',
  'Allergies'])

In [18]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ]
)


In [19]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=300, random_state=42))
])


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [21]:
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Weight_kg',
                                                   'Height_cm']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Disease_Type',
                                                   'Severity',
                                                   'Physical_Activity_Level',
                                                   'Dietary_Restrictions',
                                                   'Allergies'])])),
                ('classifier',
                 RandomForestClassifier(n_estimators=300, random_state=42))])

In [23]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

    Balanced       1.00      1.00      1.00        85
    Low_Carb       1.00      1.00      1.00        52
  Low_Sodium       1.00      1.00      1.00        63

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [25]:
joblib.dump(model,"diet_model.pkl")
print("Model saved successfully!")


Model saved successfully!
